# Test Different Evalution Functions
1. Automatically select 10 best generators
2. Generate 20 parameter sets for each generator (directly from generator)
3. Simulate bioreactor to see the final concentration
4. Calculate scoring function and find 3 most fitted ones

In [1]:
import sys
import os
# Add the parent directory to the PYTHONPATH
sys.path.append(os.path.abspath('../'))


### 2. Generate Parameter Set Using Generators

In [2]:
import numpy as np
import pandas as pd
from configparser import ConfigParser
from renaissance.evostrat.init_mlp import MLP
from renaissance.tools.helper import load_pkl


In [3]:
repeat = 10
generation = 23
cond_class = 1

lnminkm = -25
lnmaxkm = 0

pf_flag = 0
base = "../models/"
met_model = "ecoli_shikki_regulation"
names_km = load_pkl(f'{base}/{met_model}/parameters/generated_k_names.pkl')

n_sets = 20
path_to_weights = f"../output/rnsc_opt/repeat_{repeat}/weights_{generation}.pkl"
output_name = "gen_x_parameters"
ss_idx = 1712

#Parse arguments from configfile
configs = ConfigParser()
configs.read('configfile.ini')

['configfile.ini']

In [4]:
mlp = MLP(cond_class, lnminkm, lnmaxkm, n_sets, names_km, len(names_km), param_fixing=pf_flag)

# Load saved weights and generate
opt_weights = load_pkl(path_to_weights)
mlp.generator.set_weights(opt_weights)
gen_params = mlp.sample_parameters()

99


### 3. Load Model and Prepare Simulation

In [5]:
from pytfa.io.json import load_json_model
from skimpy.core import *
from skimpy.mechanisms import *
from skimpy.sampling.simple_parameter_sampler import SimpleParameterSampler
from skimpy.io.yaml import load_yaml_model
from skimpy.io.regulation import load_enzyme_regulation
from skimpy.analysis.oracle.load_pytfa_solution import load_fluxes, load_concentrations, load_equilibrium_constants
from skimpy.core.parameters import ParameterValues, ParameterValuePopulation
from sympy import Symbol
from scipy.linalg import eigvals as eigenvalues
from renaissance.kinetics.eigenvalue_calculation import calc_eigenvalues

''' Run parameters and paths '''
# Cellular parameters
CONCENTRATION_SCALING = 1e9  # 1 mol to 1 mmol
TIME_SCALING = 1  # 1hr
DENSITY = 1105  # g/L
GDW_GWW_RATIO = 0.3  # Assumes 70% Water
flux_scaling_factor = 1e-3 * (GDW_GWW_RATIO * DENSITY) * CONCENTRATION_SCALING / TIME_SCALING
NCPU = 32

In [6]:
model_file = configs['PATHS']['model_file']
thermo_experiment_file = configs['PATHS']['thermo_experiment_file']
regulation_file = configs['PATHS']['regulation_file']
kinetic_params_file = configs['PATHS']['kinetic_params_file']
steady_states_file = configs['PATHS']['steady_states_file']
steady_states_sample = 1712 #[1715,1717,2392,2482,3927,4230,4241,4456, 4468]

path_to_kmodel = f'{base}/{met_model}/kinetic/{model_file}'
path_to_tmodel = f'{base}/{met_model}/thermo/{thermo_experiment_file}'
path_to_params = f'{base}/{met_model}/parameters/gen_{repeat}_{generation}_parameters.hdf5'
path_to_regulation_data = f'{base}/{met_model}/{regulation_file}'
path_to_samples = f'{base}/{met_model}/steady_state_samples/{steady_states_file}'


In [7]:
# Load models
tmodel = load_json_model(path_to_tmodel)
kmodel_draft = load_yaml_model(path_to_kmodel)

# Add regulation data to kinetic model
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel_draft.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel_draft.reactants.keys()))]
kmodel = load_enzyme_regulation(kmodel_draft, df_regulations_all)

# Compile model
kmodel.prepare()
kmodel.compile_jacobian(ncpu=NCPU)

2024-03-13 17:51:33,324 - thermomodel_new - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2024-03-13 17:51:36,061 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies
2024-03-13 17:51:39,286 - Unnamed - WARNING - Non integer stoichiometries found ['LMPD_biomass_c_1_420', 'CYTBDpp', 'CYTBO3_4pp'] change to integer for linear dependencies


In [8]:
k_names = []
k_regulation = []

for k, v in kmodel.parameters.items():
    if k.startswith("km_") or "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
        if k not in k_names:
            k_names.append(k)
            
        if "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
            k_regulation.append(k)

print(f"The number of k for generation is {len(k_names)}, \nand the number of regulation parameters is {len(k_regulation)}.")


The number of k for generation is 464, 
and the number of regulation parameters is 106.


In [9]:
ss_idx = steady_states_sample 
samples = pd.read_csv(path_to_samples, header=0, index_col=0).iloc[ss_idx, 0:]

flux_series = load_fluxes(samples, tmodel, kmodel,
                  density=DENSITY,
                  ratio_gdw_gww=GDW_GWW_RATIO,
                  concentration_scaling=CONCENTRATION_SCALING,
                  time_scaling=TIME_SCALING)

conc_series = load_concentrations(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING)
    
# Fetch equilibrium constants
k_eq = load_equilibrium_constants(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING,
                          in_place=True)


sampling_parameters = SimpleParameterSampler.Parameters(n_samples=1)
sampler = SimpleParameterSampler(sampling_parameters)
sampler._compile_sampling_functions(kmodel, conc_series)


In [12]:
parameter_set = pd.DataFrame(np.exp(gen_params))
parameter_set.columns = names_km

param_population = []
for j in range(len(parameter_set.index)):
    param_val = parameter_set.loc[j] * CONCENTRATION_SCALING
    param_sample, eigen = calc_eigenvalues(kmodel, param_val, flux_series, conc_series, k_eq)
    param_population.append(param_sample)

param_population = ParameterValuePopulation(param_population, kmodel=kmodel)

### 3. Bioreactor Simulation

In [13]:
from skimpy.simulations.reactor import make_batch_reactor
from skimpy.core.solution import ODESolutionPopulation
from renaissance.simulation.reactor import simulate_bioreactor
import time

# Ode simulation parameters
TOTAL_TIME = 60
N_STEPS = 1000
MAX_TIME = 60
REACTOR_VOLUME = 50e13

In [14]:
# Reactor initialization
kinetic_models = list(param_population._index.keys())
reactor = make_batch_reactor('../2_bioreactor/single_species.yaml', df_regulation= df_regulations_all)
reactor.compile_ode(ncpu=NCPU, add_dilution=False)
reactor_volume = reactor.models.strain_1.parameters.strain_1_volume_e.value

2024-03-13 17:57:46,858 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [ ]:
sols_wt= []
for this_model in kinetic_models:
    print("\nRunning model: {}".format(this_model))

    # Load tfa sample and kinetic parameters into kinetic model
    parameter_set = param_population[this_model]
        
    solution, final_biomass, final_anthranilate, final_glucose = \
    simulate_bioreactor(parameter_set, 
                        samples, 
                        kmodel, 
                        conc_series, 
                        reactor, 
                        reactor_volume=REACTOR_VOLUME,
                        max_time=MAX_TIME,
                        total_time=TOTAL_TIME,
                        steps=N_STEPS)

    sols_wt.append(solution)



Running model: 0
Starting Simulation
Compelted in 5.27 seconds
-----------------
Final biomass is : 1.663765270904677, final anthranilate is : 7.215271712627369, final glucose is : 4142.774167497099

Running model: 1
Starting Simulation
Compelted in 12.90 seconds
-----------------
Final biomass is : 8.862932560193125, final anthranilate is : 14.858594890228447, final glucose is : 2.0641107401749452e-07

Running model: 2
Starting Simulation
Did not converge in time
Compelted in 60.01 seconds
-----------------
Final biomass is : 5.349112544082295, final anthranilate is : 13.82492166402992, final glucose is : 3246.42604481868

Running model: 3
Starting Simulation
Compelted in 48.68 seconds
-----------------
Final biomass is : 10.939543294694777, final anthranilate is : 13.788800613293967, final glucose is : 1.7911468664893914e-09

Running model: 4
Starting Simulation
Did not converge in time
Compelted in 60.02 seconds
-----------------
Final biomass is : 9.21941381420962, final anthranil